# Statistical Learning Model - Production


Import libraries

In [ ]:
#!conda update scikit-learn

In [18]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

import time

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split

# from sklearn.feature_extraction import TfidfVectorizer ===> gets an error
# pls use the import below
from sklearn.feature_extraction.text import TfidfVectorizer 

from scipy.sparse import hstack, vstack                            # used on bow of workds, vetoctored the 'Title' column

from sklearn.metrics import roc_auc_score, average_precision_score


%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [19]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.21.3.


## Load final raw data with labels

In [20]:
df = pd.read_csv('./raw_data/final_labels_dataset.csv', index_col=0)

print('Dataset shape before: ', df.shape)
print('Dataset columns: ', df.columns)

# drop duplicated data
df.drop_duplicates(inplace=True)

print('Dataset shape after: ', df.shape)

print('Check duplicates: ', df.duplicated().mean())
df.tail(2)

Dataset shape before:  (727, 5)
Dataset columns:  Index(['title', 'y', 'upload_date', 'view_count', 'new'], dtype='object')
Dataset shape after:  (701, 5)
Check duplicates:  0.0


,title,y,upload_date,view_count,new
1016,Data Scientists vs Data Engineers: Which one i...,1.0,20191219,183723,1.0
1468,#kaggle #DataScience Machine Learning MicroCou...,0.0,20190730,405,1.0


# 1. Data Cleanup

Using a NEW dataframe with data that's ready and clean the data to fit the ML model.

In [21]:
# create a clean dataframe with the same indice on the original dataframe - raw data
df_clean = pd.DataFrame(index=df.index)

In [22]:
df_clean['date'] = pd.to_datetime(df['upload_date'], format='%Y%m%d')

# note: format='%Y %m %d' shows the time; format='%Y%m%d' brings only YYYY-MM-DD - easy!

In [23]:
df_clean['date']

# dtype: datetime64[ns] used by numpy and pandas

0      2021-05-05
1      2021-05-05
2      2021-05-05
3      2021-05-05
4      2021-05-05
          ...    
1041   2019-11-18
488    2018-11-25
528    2018-07-24
1016   2019-12-19
1468   2019-07-30
Name: date, Length: 701, dtype: datetime64[ns]

In [24]:
# columns views: make sure all NAN will be convert to 0 and an integer data type will be added
df_clean['views'] = df['view_count'].fillna(0).astype(int64)

In [25]:
# adding title column. It will be used on the model....will be vectorized later
df_clean['title'] = df['title']

In [26]:
df_clean.dtypes

date     datetime64[ns]
views             int64
title            object
dtype: object

## 2.Features & Labels

Create an unique features dataframe. JUST an extra step. Making sure the features are ready.

**Reason**: Align the feaatures dataframe with the most cleaning data - raw data collected & cleaned. The cleaning process can skip rows or columns.


In [27]:
# features: it's similar to df_clean, just an extra step
features = pd.DataFrame(index=df_clean.index)

# labels/targets
y = df['y'].copy()

In [28]:
print('Features shape: {}'.format(shape(features)))
print('Labels shape: {}'.format(shape(y)))

Features shape: (701, 0)
Labels shape: (701,)


## Important: sklearn can't use *date* as a feature.

Let's manipulate and create a feature using the raw date - **Num_views_per_day**.

Sklearn needs a number.

In [29]:
# time_since_pub: time since the video was published. Random data choose. Use the date I created this code: fix date point - 2021-05-09

# np.timedelta64(1, 'D'): time delta in numpy. Difference in days
# we have data on a granually day, meaning a difference less than a day makes sense.
features['time_since_pub'] = (pd.to_datetime("2021-05-09") - df_clean['date']) / np.timedelta64(1, 'D')

# used features
features['views'] = df_clean['views']
features['views_per_day'] = features['views'] / features['time_since_pub']

features = features.drop(['time_since_pub'], axis=1)   # time_since_pub only used for the calculation

# time_since_pub as a feature may impact the model once the numbers seem to increase a lot and the end of the time serie.
# The training&validations datasets may not have a normal distributed values.Thus, an umbalaced feature weights
# and random samples are important to train and fit a ml model


In [30]:
features.tail()

,views,views_per_day
1041,31788,59.085502
488,65724,73.352679
528,34286,33.613725
1016,183723,362.372781
1468,405,0.624037


In [31]:
features.describe()

,views,views_per_day
count,7.010000e+02,701.000000
mean,1.525490e+05,521.359455
std,2.042103e+06,4284.958273
min,0.000000e+00,0.000000
25%,3.390000e+02,4.977941
50%,2.892000e+03,37.956522
75%,1.997200e+04,180.000000
max,4.872417e+07,95913.728346


## 3. Data Preparation

Let's try to split the train&validation datasets 50/50.

How the 2 features **view** and **views_per_day** impacted the ML model? 
Does a simple model with only 2 features impact the way the YouTube videos will be selected?

In [32]:
# check all data on df_clean
# pd.set_option('display.max_rows', 527)
# df_clean

median_date = df_clean['date'].quantile(0.5, interpolation="midpoint")
median_date

# median date 2021-03-12 before

Timestamp('2021-01-10 00:00:00')

In [34]:
# splitting features dataset - trying a 50/50 using a median date
# balanced dataset is important!!!

# code below can also be used
# Xtrain, Xval = features[df_clean['date'] < '2021-03-12'], features[df_clean['date'] >= '2021-03-12']
# ytrain, yval = y[df_clean['date'] < '2021-03-12 '], y[df_clean['date'] >= '2021-03-12 ']

# needed approach - mask parameter to select the data
mask_train = df_clean['date'] < '2021-01-10'
mask_val = df_clean['date'] >= '2021-01-10'

Xtrain, Xval = features[mask_train], features[mask_val]
ytrain, yval = y[mask_train], y[mask_val]

Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

# datasets, training & validation, not huge. But,...

((349, 2), (352, 2), (349,), (352,))

## Add the title feature

**Important**: transforming the Title string to numbers.

Building a matrix in which each column will be the counting word from the Title feature.

Import to notice that commom words like machine+learning will have a low weight.



In [35]:
title_train = df_clean[mask_train]['title']
title_val = df_clean[mask_val]['title']

# Vectorizing the Title features
title_vec = TfidfVectorizer(min_df=2, ngram_range=(1,3))   # object defined
# mind_df = 2 means the minimum numnber of words that be used to create a column
# ngram_range=(1,3) - combining words to maximum 3 words

# bow: bag of words
title_bow_train = title_vec.fit_transform(title_train)     # fit + transform: store the words on the features, plus how many times the word appeared
title_bow_val = title_vec.transform(title_val)             # validation set ONLY transform. Validation should NOT learning the words

Without ngram_range=(1,3): Shape for title bag of words matrix:  (349, 289)
with ngram_range=(1,3):   Shape for title bag of words matrix:  (349, 719)

In [36]:
# checking
print('Shape for title bag of words matrix: ', title_bow_train.shape)
title_bow_train

Shape for title bag of words matrix:  (349, 719)


<349x719 sparse matrix of type '<class 'numpy.float64'>'
	with 3968 stored elements in Compressed Sparse Row format>

 TfidfVectorizer function returns a vectorized sparse matrix. It's an optimize matrix in Scipy where only values NOT equal to zero are returned.

In [66]:
# the sparse matrix 'title_bow_train' contains 3968  elements NOT ZERO
1 - 2286 /(349*719)   
# % of ZERO elements on the sparse matrix, but only 3% are NOT ZERO elements. Meaning that the matrix is sparse computationally and mathematically speaking

0.9908899259158892

## IMPORTANT to note: 
Combining simple matrix - Xtrain&Xval - with a sparse matrix - title_bow_train & title_bow_val

Use scipy.sparse hstack and vstack

More details on hstack and vstack...stacking matrix (vectoes) horizontally and vertically

Sample:

hstack - [1 2]    [3 4]  -> [1 2 3 4]

vstack [1 2]      [3 4]  -> [1 2]
                            [3 4]
                            
USE *scipy.sparse hstack and vstack*, numpy sparse function may take TOO LONG, or not compute at all!!

In [37]:
# combining sparse matrix with original features
from scipy.sparse import hstack, vstack  

Xtrain_wtitle = hstack([Xtrain, title_bow_train])
Xval_wtitle = hstack([Xval, title_bow_val])

In [38]:
Xtrain_wtitle.shape, Xval_wtitle.shape

# 2 nummerical features on training dataset plus 289 columns from 'Title'

((349, 721), (352, 721))

## Random Forest

In [69]:
# check number of 1 samples under train dataset
print('Positive samples - videos select: {}'.format(ytrain.mean() * 349))
print(' % of positive samples - videos select: {}'.format(ytrain.mean() * 100))

# definitely unbalaced

Positive samples - videos select: 107.0
 % of positive samples - videos select: 30.659025787965614


The training dataset is not big!!

In [70]:
clf_rf = RandomForestClassifier(n_estimators=1000, random_state=0, min_samples_leaf=1, class_weight='balanced', n_jobs=6)    # defined object

## Fitting the model against the train dataset

NOW: 3 features - views, views per day, and title

In [71]:
clf_rf.fit(Xtrain_wtitle, ytrain)

C:\Users\danie\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:489: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_store_unique_indices = np.zeros(y.shape, dtype=np.int)
C:\Users\danie\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your curr

RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=1000, n_jobs=6, oob_score=False,
                       random_state=0, verbose=0, warm_start=False)

In [52]:
print('ML model already trainded/fitted and ready to be used!')

ML model already trainded/fitted and ready to be used!


## Predicting if a video has been select

Probability = 1

predict_proba: returns a numpy array with prob of zero and prob of 1

In [72]:
pred = clf_rf.predict_proba(Xval_wtitle)[:, 1]   # only 1

C:\Users\danie\Anaconda3\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)


## Metrics - validating the model


 Before: Random Forest Model

average precision: 0.4566659359075169

Roc Auc: 0.5182456140350877

In [73]:
# area of precision for decision tree
print('Random Forest baseline model - average precision')
average_precision_score(yval, pred)

Random Forest baseline model - average precision


0.4605227155328958

## IMPORTANT: any future model in PRD should have a greater than baseline model **0.50**

In [74]:
# area under curve of roc curve metric
print('Random Forest baseline model - roc auc')
roc_auc_score(yval, pred)

Random Forest baseline model - roc auc


0.5253960396039604

## Validating metrics before and after the active learning samples

Increassing the validation dataset metrics:

roc auc: 0.47

average precision: 0.41

----------------------------------------------------------
Original decision metric - baseline model

roc auc: 0.49

average precision: 0.42

---------------------------------------------------------

Metrics seem to be on the variation range. Meaning AP & ROC AUC are close enough between baseline model and new model with active learning samples.
Please note that the number of samples are not significantly big, then the metrics might not change a lot...small steps definitely count!!!

---------------------------------------------------------

## Productin Baseline Model
roc auc: 0.46

average precision: 0.52

**Details**: TfidfVectorizer(min_df=2, ngram_range=(1,3)) 
---------------------------------------------------------


## LigthGBM Model

In [39]:
cl_lgbm = LGBMClassifier(random_state=0, class_weight='balanced', n_jobs=6)   # defined object

In [40]:
cl_lgbm.fit(Xtrain_wtitle, ytrain)

LGBMClassifier(boosting_type='gbdt', class_weight='balanced',
               colsample_bytree=1.0, importance_type='split', learning_rate=0.1,
               max_depth=-1, min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=100, n_jobs=6, num_leaves=31,
               objective=None, random_state=0, reg_alpha=0.0, reg_lambda=0.0,
               silent=True, subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0)

In [41]:
pred = cl_lgbm.predict_proba(Xval_wtitle)[:,1]

C:\Users\danie\Anaconda3\lib\site-packages\lightgbm\basic.py:739: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


In [42]:
# area of precision for decision tree
print('Random Forest baseline model - average precision: ', average_precision_score(yval, pred))

# area under curve of roc curve metric
print('Random Forest baseline model - roc auc: ', roc_auc_score(yval, pred))

Random Forest baseline model - average precision:  0.4052786105139529
Random Forest baseline model - roc auc:  0.48495049504950494


# Bayesian Optmization

In [44]:
from skopt import forest_minimize

In [46]:
def tune_lgbm(params):
    print(params)
    lr = params[0]
    max_depth = params[1]
    min_child_samples = params[2]
    subsample = params[3]
    colsample_bytree = params[4]
    n_estimators = params[5]
    
    min_df = params[6]
    ngram_range = (1, params[7])
    
    title_vec = TfidfVectorizer(min_df=min_df, ngram_range=ngram_range)
    title_bow_train = title_vec.fit_transform(title_train)
    title_bow_val = title_vec.transform(title_val)
    
    Xtrain_wtitle = hstack([Xtrain, title_bow_train])
    Xval_wtitle = hstack([Xval, title_bow_val])
    
    mdl = LGBMClassifier(learning_rate=lr, num_leaves=2 ** max_depth, max_depth=max_depth, 
                         min_child_samples=min_child_samples, subsample=subsample,
                         colsample_bytree=colsample_bytree, bagging_freq=1,n_estimators=n_estimators, random_state=0, 
                         class_weight="balanced", n_jobs=6)
    mdl.fit(Xtrain_wtitle, ytrain)
    
    p = mdl.predict_proba(Xval_wtitle)[:, 1]
    
    print(roc_auc_score(yval, p))
    
    return -average_precision_score(yval, p)

space = [(1e-3, 1e-1, 'log-uniform'), # lr    # learning rate
          (1, 10), # max_depth                # tree depth (20 was trowing an error..too deep for this problem). Maybe because there are few samples
          (1, 20), # min_child_samples
          (0.05, 1.), # subsample
          (0.05, 1.), # colsample_bytree
          (100,1000), # n_estimators          # numb of trees
          (1,5), # min_df                     # num of words on the title
          (1,5)] # ngram_range                # num of words grouped

res = forest_minimize(tune_lgbm, space, random_state=160745, n_random_starts=20, n_calls=100, verbose=1)


Iteration No: 1 started. Evaluating function at random point.
[0.009944912110647982, 5, 1, 0.4677107511929402, 0.49263223036174764, 272, 3, 1]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


C:\Users\danie\Anaconda3\lib\site-packages\lightgbm\basic.py:739: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


0.4867161716171617
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.4352
Function value obtained: -0.4137
Current minimum: -0.4137
Iteration No: 2 started. Evaluating function at random point.
[0.053887464791860025, 1, 15, 0.7437489153990157, 0.8675167974293533, 549, 3, 4]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


C:\Users\danie\Anaconda3\lib\site-packages\lightgbm\basic.py:739: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


0.47650165016501655
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.2236
Function value obtained: -0.4070
Current minimum: -0.4137
Iteration No: 3 started. Evaluating function at random point.
[0.004151454520895999, 6, 20, 0.8682075103820793, 0.9491436163200662, 411, 4, 3]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


...

0.5193234323432343
Iteration No: 98 ended. Search finished for the next optimal point.
Time taken: 1.7665
Function value obtained: -0.4447
Current minimum: -0.4709
Iteration No: 99 started. Searching for the next optimal point.
[0.0012022803873517154, 1, 4, 0.9648837901288677, 0.12817269323992408, 891, 4, 5]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


C:\Users\danie\Anaconda3\lib\site-packages\lightgbm\basic.py:739: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


0.5267491749174917
Iteration No: 99 ended. Search finished for the next optimal point.
Time taken: 0.9666
Function value obtained: -0.4443
Current minimum: -0.4709
Iteration No: 100 started. Searching for the next optimal point.
[0.0015158247975690623, 9, 9, 0.8908223182660441, 0.34840088282954385, 405, 4, 5]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


C:\Users\danie\Anaconda3\lib\site-packages\lightgbm\basic.py:739: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


0.5127557755775577
Iteration No: 100 ended. Search finished for the next optimal point.
Time taken: 1.2166
Function value obtained: -0.4393
Current minimum: -0.4709


## LGBM metrics
average precision: 0.4709

auc:  0.5155610561056105

Optmized parameters:  [0.0026544900377379554, 5, 7, 0.967093260484254, 0.07552826835375895, 143, 4, 5]

## Get optmize parameters

In [49]:
print('Optmized parameters: ', res.x)

Optmized parameters:  [0.0026544900377379554, 5, 7, 0.967093260484254, 0.07552826835375895, 143, 4, 5]


Learning Rate: 0.0026544900377379554

Max depth: 5

Min child samples: 7

Subsample: 0.967093260484254

Col Samples by tree: 0.07552826835375895  (% of samples used in each tree(

Num of trees (n_estimators): 143

min_df: 4

ngram_range: 5

## Logistic Regression


In [50]:
from sklearn.preprocessing import MaxAbsScaler, StandardScaler
from scipy.sparse import csr_matrix

In [55]:
Xtrain_wtitle2 = csr_matrix(Xtrain_wtitle.copy())
Xval_wtitle2 = csr_matrix(Xval_wtitle.copy())

#scaler = StandardScaler()
scaler = MaxAbsScaler()


#Xtrain_wtitle2[:, :2] = scaler.fit_transform(Xtrain_wtitle2[:, :2].todense())
#Xval_wtitle2[:, :2] = scaler.transform(Xval_wtitle2[:, :2].todense())

Xtrain_wtitle2 = scaler.fit_transform(Xtrain_wtitle2)
Xval_wtitle2 = scaler.transform(Xval_wtitle2)

In [54]:
Xval_wtitle2.shape

(352, 721)

In [56]:
cl_lr = LogisticRegression(C=0.5,n_jobs=6, random_state=0)
cl_lr.fit(Xtrain_wtitle2, ytrain)

C:\Users\danie\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\danie\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 6.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


LogisticRegression(C=0.5, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=6, penalty='l2', random_state=0,
                   solver='warn', tol=0.0001, verbose=0, warm_start=False)

In [57]:
p = cl_lr.predict_proba(Xval_wtitle2)[:, 1]

In [58]:
average_precision_score(yval, p), roc_auc_score(yval, p)

(0.46816171307972243, 0.5476402640264026)